### joblib实现并行运算的思想是将一组通过循环产生的串行计算子任务，以多进程或多线程的方式进行调度，而我们针对自定义的运算任务需要做的仅仅是将它们封装为函数的形式即可，譬如：

In [7]:
import time
def task_demo1(i):
    time.sleep(1)
    return i,time.time()

### 接着只需要像下面的形式一样，为Parallel()设置相关参数后，衔接循环创建子任务的列表推导过程，其中利用delayed()包裹自定义任务函数，再衔接()传递任务函数所需的参数即可，其中n_jobs参数用于设置并行任务同时执行的worker数量，因此在这个例子中可以看到进度条是按照4个一组递增的，可以看到最终时间开销也达到了并行加速效果：

In [8]:
from tqdm.notebook import tqdm
from joblib import Parallel,delayed

results=(
    Parallel(n_jobs=4)
    (delayed(task_demo1)(i)
     for i in tqdm(range(20)))
)
results

  0%|          | 0/20 [00:00<?, ?it/s]

[(0, 1653309288.482769),
 (1, 1653309288.482777),
 (2, 1653309288.483407),
 (3, 1653309288.483426),
 (4, 1653309289.519325),
 (5, 1653309289.5422678),
 (6, 1653309289.543654),
 (7, 1653309289.551724),
 (8, 1653309290.522603),
 (9, 1653309290.5469382),
 (10, 1653309290.546985),
 (11, 1653309290.5557098),
 (12, 1653309291.5268538),
 (13, 1653309291.551776),
 (14, 1653309291.553122),
 (15, 1653309291.5570738),
 (16, 1653309292.529977),
 (17, 1653309292.556055),
 (18, 1653309292.556066),
 (19, 1653309292.563607)]

### 其中可以根据计算任务以及机器CPU核心数具体情况为Parallel()调节参数，核心参数有：

* **backend：用于设置并行方式，其中多进程方式有'loky'（更稳定）和'multiprocessing'两种可选项，多线程有'threading'一种选项。默认为'loky'**
* **n_jobs：用于设置并行任务同时执行的worker数量，当并行方式为多进程时，n_jobs最多可设置为机器CPU逻辑核心数量，超出亦等价于开启全部核心，你也可以设置为-1来快捷开启全部逻辑核心，若你不希望全部CPU资源均被并行任务占用，则可以设置更小的负数来保留适当的空闲核心，譬如设 置为-2则开启全部核心-1个核心，设置为-3则开启全部核心-2个核心**
　　譬如下面的例子，在我这台逻辑核心数为4的机器上，保留两个核心进行并行计算：

In [9]:
import os
os.cpu_count()

4

In [10]:
results=(
    Parallel(n_jobs=-2)
    (delayed(task_demo1)(i)
     for i in tqdm(range(20)))
)
results

  0%|          | 0/20 [00:00<?, ?it/s]

[(0, 1653309546.4542131),
 (1, 1653309546.470762),
 (2, 1653309546.4819882),
 (3, 1653309547.458091),
 (4, 1653309547.4742022),
 (5, 1653309547.4868312),
 (6, 1653309548.463306),
 (7, 1653309548.480268),
 (8, 1653309548.492597),
 (9, 1653309549.465745),
 (10, 1653309549.484669),
 (11, 1653309549.4982872),
 (12, 1653309550.471017),
 (13, 1653309550.4879959),
 (14, 1653309550.501437),
 (15, 1653309551.474784),
 (16, 1653309551.493016),
 (17, 1653309551.504429),
 (18, 1653309552.478865),
 (19, 1653309552.497494)]

### 关于并行方式的选择上，由于Python中多线程时全局解释器锁的限制，如果你的任务是计算密集型，则推荐使用默认的多进程方式加速，如果你的任务是IO密集型譬如文件读写、网络请求等，则多线程是更好的方式且可以将n_jobs设置的很大，举个简单的例子，可以看到，通过多线程并行，我们在5秒的时间里完成了1000次请求，远快于单线程17秒请求100次的成绩：

In [11]:
import requests
def task_demo2():
    response=requests.get('https://www.baidu.com/')
    return response.status_code

In [12]:
#常规单线程进行
for i in tqdm(range(100)):
    task_demo2()

  0%|          | 0/100 [00:00<?, ?it/s]

In [19]:
results=(
    Parallel(n_jobs=50,backend='threading')
    (delayed(task_demo2)()
     for i in tqdm(range(1000)) )
)

  0%|          | 0/1000 [00:00<?, ?it/s]

ConnectionError: HTTPSConnectionPool(host='www.baidu.com', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x12294b670>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [17]:
results

[200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200,
 200]